# Healthcare Fintech Lending Funnel Metrics & Data Quality

**Role angle:** Healthcare fintech data quality, product & analytics infrastructure  

This notebook walks through how I would audit and model a multi‑system patient financing funnel, define shared business concepts, validate the data, and deliver trustworthy funnel, provider, and vendor‑level reporting.  
The goal is to simulate a contract engagement where the analyst is responsible for data definitions, modeling, QA/UAT, and dashboards across patient applications, financing plans, marketing, and product events for healthcare providers.


In [ ]:
# 0. Setup: imports & database connection

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter display settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
sns.set(style="whitegrid")

# ---- DATABASE CONNECTION (adjust for your environment) ----
import psycopg2
from psycopg2.extras import RealDictCursor

def get_connection():
    """
    Update with your own credentials or environment variables.
    """
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="analytics",
        user="your_user",
        password="your_password",
    )
    return conn



def run_query(sql, params=None):
    """
    Helper to run a query and return a pandas DataFrame.
    """
    conn = get_connection()
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, params or {})
            rows = cur.fetchall()
        df = pd.DataFrame(rows)
        return df
    finally:
        conn.close()


## 1. Business Problem

A healthcare fintech company helps patients finance medical bills and procedures through payment plans offered in partnership with hospitals and clinics.  
They work with multiple systems:

- A financing application system for patient applications.  
- A servicing platform that tracks payment plans and loan status.  
- Marketing platforms (paid search/social, provider referrals, affiliates).  
- Product analytics tools (e.g., Heap, Amplitude) that track patient behavior across the application funnel.  

Today, each system reports slightly different numbers for "applications", "approvals", and "funded plans", and provider and vendor performance, making it difficult for stakeholders to trust funnel metrics used in provider negotiations and growth planning.

Stakeholders need to answer:

- How many patients start, submit, and fund a financing application over time, by provider and channel?  
- Where are the biggest drop‑offs in the funnel, and how do they vary by provider, service line, vendor, and channel?  
- Which partners and marketing channels drive high‑quality, funded payment plans rather than just clicks or incomplete applications?

This project defines a shared data model and metric layer for the patient financing funnel, validates data quality across systems, and produces funnel and provider/vendor‑level dashboards that Engineering, Product, Growth, and Provider Partnerships can trust.



## 2. Data Understanding

We work with four raw datasets:

- `raw_applications`: one row per patient financing application, with metadata, provider context, and statuses.  
- `raw_loans`: one row per funded financing plan, tied back to an application and patient.  
- `raw_marketing`: paid marketing touches and costs (including provider‑referral and digital channels).  
- `raw_events`: product analytics‑style events representing patient interactions across the financing funnel.

In this section, I profile each table to understand grain, key fields, and quirks that will influence modeling, healthcare context, and validation.

In [ ]:
sql = """
SELECT *
FROM raw_applications
LIMIT 5;
"""
apps_sample = run_query(sql)
apps_sample

In [ ]:
# Basic profiling for applications
sql = """
SELECT
    COUNT(*)                          AS row_count,
    COUNT(DISTINCT application_id)    AS distinct_applications,
    COUNT(DISTINCT customer_id)       AS distinct_customers,
    MIN(created_at)                   AS min_created_at,
    MAX(created_at)                   AS max_created_at,
    COUNT(*) FILTER (WHERE status = 'approved') AS approved_count,
    COUNT(*) FILTER (WHERE status = 'rejected') AS rejected_count,
    COUNT(*) FILTER (WHERE status = 'withdrawn') AS withdrawn_count
FROM raw_applications;
"""
run_query(sql)


### Observations – `raw_applications`

- **Grain:** one row per `application_id` (confirmed by uniqueness check).  
- **Primary identifiers:** `application_id`, with `customer_id` as a foreign key to the customer/patient domain.  
- **Time:** `created_at` gives us a natural application date and allows cohorting over time.  
- **Business nuance:** `status` may not directly map to "approved" or "funded" – we will later derive funnel outcomes using both this table and `raw_loans`.  
- **Potential quirks:** multiple applications per customer/patient, and possible status transitions not captured in a single row (e.g., pending → approved → funded).


In [ ]:
sql = """
SELECT *
FROM raw_loans
LIMIT 5;
"""
loans_sample = run_query(sql)
loans_sample


In [ ]:
# Profiling loans
sql = """
SELECT
    COUNT(*)                          AS row_count,
    COUNT(DISTINCT loan_id)           AS distinct_loans,
    COUNT(DISTINCT application_id)    AS distinct_applications,
    MIN(funded_at)                    AS min_funded_at,
    MAX(funded_at)                    AS max_funded_at,
    COUNT(*) FILTER (WHERE status = 'charged_off') AS charged_off_count,
    COUNT(*) FILTER (WHERE status = 'paid_off')     AS paid_off_count
FROM raw_loans;
"""
run_query(sql)


### Observations – `raw_loans`

- **Grain:** one row per `loan_id`, linked to `application_id` and `customer_id`.  
- **Business nuance:** a loan may be charged off or paid off; this will drive default rate and performance metrics.  
- **Join behavior:** we expect every loan to have a corresponding application; any mismatches will surface in validation.  
- **Time:** `funded_at` will anchor our "funded loans per month" and vendor performance views.

In [ ]:
sql = """
SELECT *
FROM raw_marketing
LIMIT 5;
"""
mkt_sample = run_query(sql)
mkt_sample

In [ ]:
# Profiling marketing touches
sql = """
SELECT
    COUNT(*)                             AS row_count,
    COUNT(DISTINCT marketing_touch_id)   AS distinct_touches,
    COUNT(DISTINCT application_id)       AS distinct_applications_touched,
    COUNT(DISTINCT campaign_id)          AS distinct_campaigns,
    COUNT(DISTINCT channel)              AS distinct_channels,
    COUNT(DISTINCT vendor)               AS distinct_vendors,
    SUM(cost_usd)                        AS total_spend
FROM raw_marketing;
"""
run_query(sql)

### Observations – `raw_marketing`

- **Grain:** one row per marketing touch (`marketing_touch_id`), which we will roll up to the application level for attribution.  
- **Vendors & channels:** `vendor` and `channel` identify third‑party products (Google, Meta, partners) and tactics.  
- **Attribution assumption:** in this project we will use a simple **first‑touch at application** model; future work could explore different attribution windows and multi‑touch models.  
- **Data quality risk:** some touches may not be tied to an `application_id` (upper‑funnel activity) or may tie to multiple applications per customer/patient; these cases need explicit decisions in metric definitions.


In [ ]:
sql = """
SELECT *
FROM raw_events
LIMIT 5;
"""
events_sample = run_query(sql)
events_sample


In [ ]:
# Profiling events
sql = """
SELECT
    COUNT(*)                          AS row_count,
    COUNT(DISTINCT event_id)          AS distinct_events,
    COUNT(DISTINCT event_name)        AS distinct_event_names,
    COUNT(DISTINCT source_system)     AS distinct_sources,
    MIN(event_timestamp)              AS min_ts,
    MAX(event_timestamp)              AS max_ts
FROM raw_events;
"""
run_query(sql)

In [ ]:
# Distribution of event_name
sql = """
SELECT
    event_name,
    COUNT(*) AS event_count
FROM raw_events
GROUP BY event_name
ORDER BY event_count DESC;
"""
run_query(sql)

### Observations – `raw_events`

- **Grain:** one row per `event_id`, an event stream similar to what Heap/Amplitude would generate.  
- **Key funnel events:** e.g., `page_view_application`, `application_started`, `application_submitted`. We’ll use these to construct funnel flags in the applications fact table.  
- **Source systems:** events may come from different tools (`source_system`), which can create conflicting counts; our modeled fact table will normalize them.  
- **Event gaps:** missing `application_id` or inconsistent tagging can cause funnels to under‑count; we will surface these in QA.

### Clinical / Provider Fields & Mapping

Because this is a healthcare financing context, applications may include basic clinical and provider fields such as:

- `provider_id` – the hospital or clinic associated with the financing application.  
- `service_line` – broad clinical area (e.g., cardiology, orthopedics, dermatology).  
- `icd10_code` – an anonymized diagnosis or procedure code used for reporting.  

For a data migration or mapping project, it’s important to identify unmapped or invalid codes and provider IDs so they can be researched and cleaned before go‑live.


In [ ]:
# Profile ICD-10 codes and provider IDs (if present)
sql_clinical_profile = """
SELECT
    COUNT(*)                          AS row_count,
    COUNT(DISTINCT provider_id)       AS distinct_providers,
    COUNT(DISTINCT service_line)      AS distinct_service_lines,
    COUNT(DISTINCT icd10_code)        AS distinct_icd10_codes
FROM raw_applications;
"""
run_query(sql_clinical_profile)


In [ ]:
# Example unmapped ICD-10 codes (if you have a mapping table)
sql_unmapped_icd10 = """
SELECT
    a.icd10_code,
    COUNT(*) AS application_count
FROM raw_applications a
LEFT JOIN icd10_mapping m
    ON a.icd10_code = m.raw_icd10_code
WHERE a.icd10_code IS NOT NULL
  AND m.normalized_icd10_code IS NULL
GROUP BY a.icd10_code
ORDER BY application_count DESC;
"""
unmapped_icd10_df = run_query(sql_unmapped_icd10)
unmapped_icd10_df.head()

### Observations – Clinical Mapping

- This view surfaces any ICD-10 codes that are not yet mapped to a normalized code or description.  
- In a real migration, these rows would be reviewed with clinical or coding SMEs, and the mapping table updated to cover valid codes while flagging true errors.  
- A similar pattern can be applied to provider IDs and service lines to catch issues before hospital or health-system clients are migrated.

This mirrors the type of ad-hoc-but-repeatable mapping and “one-off” resolution work described in healthcare data migration roles.

## 3. Data Modeling

The goal of modeling is to create a **stable metrics layer** that:

- Has a clear grain and primary key for each table.  
- Organizes data into subject areas (Applications, Loans/Servicing, Marketing, Events).  
- Makes funnel and vendor‑level reporting repeatable and trustworthy.

I use a star‑schema‑inspired approach:

- Shared dimensions: `dim_date`, `dim_customer`, `dim_loan`.  
- Fact tables: `fct_applications`, `fct_marketing_attribution`, `fct_funnel_events`.

In [ ]:
# Create dim_date
sql_dim_date = """
CREATE TABLE IF NOT EXISTS dim_date AS
SELECT
    d::date                     AS date_key,
    EXTRACT(YEAR  FROM d)       AS year,
    EXTRACT(MONTH FROM d)       AS month,
    EXTRACT(DAY   FROM d)       AS day,
    EXTRACT(QUARTER FROM d)     AS quarter
FROM generate_series(
    DATE '2024-01-01',
    DATE '2026-12-31',
    INTERVAL '1 day'
) AS d;
"""
_ = run_query(sql_dim_date)

# Create dim_customer
sql_dim_customer = """
CREATE TABLE IF NOT EXISTS dim_customer AS
SELECT
    customer_id,
    MIN(created_at)                       AS first_seen_at,
    COUNT(DISTINCT application_id)        AS lifetime_applications
FROM raw_applications
GROUP BY customer_id;
"""
_ = run_query(sql_dim_customer)

# Create dim_loan
sql_dim_loan = """
CREATE TABLE IF NOT EXISTS dim_loan AS
SELECT
    l.loan_id,
    l.application_id,
    l.customer_id,
    l.vendor,
    l.principal_amount,
    l.interest_rate,
    l.term_months,
    l.status,
    DATE(l.funded_at)         AS funded_date
FROM raw_loans l;
"""
_ = run_query(sql_dim_loan)

### Why this dimensional structure?

- **dim_date** lets us roll up any metric to day, month, quarter, and year without recomputing date logic each time.  
- **dim_customer** captures reusable customer‑level/patient-level attributes (first seen date, application count) for lifecycle analysis.  
- **dim_loan** centralizes loan attributes and aligns them with applications and customers/patients, enabling servicing and performance views without duplicating columns across facts.

In [ ]:
sql_fct_applications = """
CREATE TABLE IF NOT EXISTS fct_applications AS
WITH app_base AS (
    SELECT
        a.application_id,
        a.customer_id,
        a.created_at,
        DATE(a.created_at)                    AS application_date,
        a.product_type,
        a.source_system,
        a.status                               AS application_status,
        a.requested_amount,
        a.term_months,
        a.channel,
        a.vendor                               AS application_vendor
    FROM raw_applications a
),

loan_flags AS (
    SELECT
        application_id,
        COUNT(*) > 0                               AS has_loan,
        MAX(CASE WHEN status = 'charged_off' THEN 1 ELSE 0 END) AS has_default,
        MAX(funded_at)                             AS max_funded_at
    FROM raw_loans
    GROUP BY application_id
),

funnel_flags AS (
    SELECT
        application_id,
        MAX(CASE WHEN event_name = 'page_view_application' THEN 1 ELSE 0 END) AS saw_application_page,
        MAX(CASE WHEN event_name = 'application_started' THEN 1 ELSE 0 END)  AS application_started,
        MAX(CASE WHEN event_name = 'application_submitted' THEN 1 ELSE 0 END) AS application_submitted
    FROM raw_events
    GROUP BY application_id
)

SELECT
    ab.*,
    lf.has_loan,
    lf.has_default,
    lf.max_funded_at,
    ff.saw_application_page,
    ff.application_started,
    ff.application_submitted
FROM app_base ab
LEFT JOIN loan_flags   lf ON ab.application_id = lf.application_id
LEFT JOIN funnel_flags ff ON ab.application_id = ff.application_id;
"""
_ = run_query(sql_fct_applications)

### Why this fact table design?

- **Grain:** one row per `application_id` – this is the core business entity the funnel revolves around.  
- **Funnel flags:** booleans for key steps (page view, start, submit) derived from events. This makes funnel analysis a simple `SUM()` over flags instead of fragile string filters.  
- **Outcome fields:** `has_loan`, `has_default`, and `max_funded_at` summarize downstream loan outcomes at the application level, simplifying reporting and attribution.  
- This structure aligns with how product analytics tools represent funnels, but in a SQL‑friendly form suitable for BI and data quality checks.

In [ ]:
# fct_marketing_attribution
sql_fct_marketing = """
CREATE TABLE IF NOT EXISTS fct_marketing_attribution AS
WITH ranked_touches AS (
    SELECT
        mt.marketing_touch_id,
        mt.customer_id,
        mt.application_id,
        mt.campaign_id,
        mt.channel,
        mt.vendor,
        mt.click_timestamp,
        mt.cost_usd,
        ROW_NUMBER() OVER (
            PARTITION BY mt.application_id
            ORDER BY mt.click_timestamp ASC
        ) AS rn
    FROM raw_marketing mt
),

first_touch AS (
    SELECT *
    FROM ranked_touches
    WHERE rn = 1
)

SELECT
    ft.application_id,
    ft.customer_id,
    ft.campaign_id,
    ft.channel,
    ft.vendor,
    ft.click_timestamp,
    ft.cost_usd                           AS first_touch_cost_usd
FROM first_touch ft;
"""
_ = run_query(sql_fct_marketing)

# fct_funnel_events
sql_fct_events = """
CREATE TABLE IF NOT EXISTS fct_funnel_events AS
SELECT
    e.event_id,
    e.customer_id,
    e.session_id,
    e.application_id,
    e.event_name,
    e.event_timestamp,
    DATE(e.event_timestamp)          AS event_date,
    e.source_system,
    e.device,
    e.url_path
FROM raw_events e;
"""
_ = run_query(sql_fct_events)

### Attribution and event facts

- **fct_marketing_attribution** implements a simple first‑touch attribution at the application level, which is easy to explain to non‑technical stakeholders and a good starting point for partner/vendor reporting.  
- **fct_funnel_events** keeps the raw event stream accessible for deeper product analytics (step sequences, pathing) while the application fact table uses summarized flags for standard funnel KPIs.


## 4. Validation & UAT

Before trusting any metric or dashboard, I validate:

1. Row‑level integrity between raw and modeled tables.  
2. Referential integrity across domains (applications ↔ loans ↔ events).  
3. Funnel logic consistency (no impossible states like "funded but not submitted").  
4. Metric reconciliation between raw counts and derived facts.

This section runs targeted SQL checks and summarizes the results.

In [ ]:
sql_counts = """
SELECT
    'applications' AS entity,
    (SELECT COUNT(*) FROM raw_applications) AS raw_count,
    (SELECT COUNT(*) FROM fct_applications) AS fact_count
UNION ALL
SELECT
    'loans' AS entity,
    (SELECT COUNT(*) FROM raw_loans) AS raw_count,
    (SELECT COUNT(*) FROM dim_loan)  AS fact_count;
"""
counts_df = run_query(sql_counts)
counts_df

**Check 1 – Raw vs fact counts**

- We expect `fct_applications` to have one row per `raw_applications` record.  
- We expect `dim_loan` to have one row per `raw_loans` record.  

Any discrepancy here indicates either filtered‑out rows or duplicates that must be explained and documented.


In [ ]:
sql_ref_integrity = """
-- Loans that don't map to an application
SELECT
    'loans_with_missing_application' AS check_name,
    COUNT(*) AS record_count
FROM raw_loans l
LEFT JOIN fct_applications fa
    ON l.application_id = fa.application_id
WHERE fa.application_id IS NULL

UNION ALL

-- Events that reference non-existent applications
SELECT
    'events_with_missing_application' AS check_name,
    COUNT(*) AS record_count
FROM raw_events e
LEFT JOIN fct_applications fa
    ON e.application_id = fa.application_id
WHERE e.application_id IS NOT NULL
  AND fa.application_id IS NULL;
"""
ref_df = run_query(sql_ref_integrity)
ref_df

**Check 2 – Referential integrity**

- **Loans with missing applications:** ideally this is zero; if not, it suggests data ingestion gaps or legacy applications not represented in the current system.  
- **Events with missing applications:** a non‑zero count is common (upper funnel traffic) but must be acknowledged; events without an application cannot contribute to application‑level metrics and should be excluded from funnel denominators.

These findings feed into the “known limitations” section of the data dictionary.

In [ ]:
sql_funnel_sanity = """
SELECT
    'funded_without_submit' AS check_name,
    COUNT(*) AS application_count
FROM fct_applications
WHERE has_loan = TRUE
  AND COALESCE(application_submitted, 0) = 0;
"""
funnel_sanity_df = run_query(sql_funnel_sanity)
funnel_sanity_df

**Check 3 – Funnel sanity**

- **Funded without submit:** in a clean funnel, this should be very close to zero.  
- If there are non‑zero counts, I investigate whether:
  - Some applications are created programmatically (e.g., internal tools).  
  - Events are missing for specific channels/vendors.  

Any systematic pattern (e.g., particular partner) becomes a data quality issue we flag to Engineering/Product.

In [ ]:
sql_vendor_recon = """
WITH raw_agg AS (
    SELECT
        vendor,
        COUNT(*) AS raw_approved
    FROM raw_applications
    WHERE status = 'approved'
    GROUP BY vendor
),
fact_agg AS (
    SELECT
        application_vendor AS vendor,
        COUNT(*) AS fact_approved
    FROM fct_applications
    WHERE application_status = 'approved'
    GROUP BY application_vendor
)

SELECT
    COALESCE(r.vendor, f.vendor) AS vendor,
    COALESCE(r.raw_approved, 0)  AS raw_approved,
    COALESCE(f.fact_approved, 0) AS fact_approved,
    COALESCE(f.fact_approved, 0) - COALESCE(r.raw_approved, 0) AS diff
FROM raw_agg r
FULL OUTER JOIN fact_agg f
    ON r.vendor = f.vendor
ORDER BY vendor;
"""
vendor_recon_df = run_query(sql_vendor_recon)
vendor_recon_df

**Check 4 – Vendor‑level reconciliation**

This check ensures that vendor‑level approval counts match between:

- A naive aggregation directly on `raw_applications`.  
- The derived `fct_applications` fact table.

Ideally `diff` is zero for all vendors. Any mismatch signals transformation bugs (e.g., filters, joins) that must be fixed before stakeholders use vendor dashboards for partner negotiations.

## 5. Analysis & Insights

With the modeled tables validated, we can now answer core funnel and vendor questions in a way that is:

- **Consistent:** definitions are centralized in the fact/dim tables.  
- **Explainable:** we can trace each metric back to source tables and QA checks.  
- **Actionable:** visualizations highlight where Product and Growth should focus.

In [ ]:
sql_funnel_by_month = """
WITH app_enriched AS (
    SELECT
        application_id,
        DATE_TRUNC('month', application_date)         AS month,
        application_vendor,
        channel,
        application_started,
        application_submitted,
        CASE WHEN application_status = 'approved' THEN 1 ELSE 0 END AS is_approved,
        CASE WHEN has_loan THEN 1 ELSE 0 END          AS is_funded
    FROM fct_applications
)
SELECT
    month,
    COUNT(*)                                  AS applications,
    SUM(application_started)                  AS started,
    SUM(application_submitted)                AS submitted,
    SUM(is_approved)                          AS approved,
    SUM(is_funded)                            AS funded
FROM app_enriched
GROUP BY month
ORDER BY month;
"""
funnel_month_df = run_query(sql_funnel_by_month)
funnel_month_df

In [ ]:
# Compute conversion rates and visualize
funnel_month_df['start_rate'] = funnel_month_df['started'] / funnel_month_df['applications']
funnel_month_df['submit_rate'] = funnel_month_df['submitted'] / funnel_month_df['started']
funnel_month_df['approve_rate'] = funnel_month_df['approved'] / funnel_month_df['submitted']
funnel_month_df['fund_rate'] = funnel_month_df['funded'] / funnel_month_df['approved']

funnel_month_df

In [ ]:
plt.figure(figsize=(10, 5))
for col, label in [
    ('start_rate', 'Start rate'),
    ('submit_rate', 'Submit rate (of starters)'),
    ('approve_rate', 'Approve rate (of submitters)'),
    ('fund_rate', 'Fund rate (of approvals)'),
]:
    plt.plot(funnel_month_df['month'], funnel_month_df[col], marker='o', label=label)

plt.title("Funnel Conversion Rates by Month")
plt.xlabel("Month")
plt.ylabel("Rate")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Insight – Funnel conversion over time

- This view shows where the biggest drop‑offs occur in the **patient financing funnel** and whether they are stable or deteriorating over time.  
- For example, if **submit rate** is declining while start rate is stable, Product and Operations might need to investigate UX friction or process issues in the online financing application (e.g., required clinical or billing fields, confusing consent screens).  
- If **approve rate** trends down, that could indicate changes in underwriting rules, shifts in patient mix from certain providers or service lines, or data quality issues with key fields (in

In [ ]:
sql_funnel_by_vendor = """
WITH app_enriched AS (
    SELECT
        application_id,
        application_vendor,
        channel,
        application_started,
        application_submitted,
        CASE WHEN application_status = 'approved' THEN 1 ELSE 0 END AS is_approved,
        CASE WHEN has_loan THEN 1 ELSE 0 END          AS is_funded
    FROM fct_applications
)
SELECT
    application_vendor AS vendor,
    COUNT(*)                       AS applications,
    SUM(application_started)       AS started,
    SUM(application_submitted)     AS submitted,
    SUM(is_approved)               AS approved,
    SUM(is_funded)                 AS funded
FROM app_enriched
GROUP BY application_vendor
HAVING COUNT(*) >= 50   -- Only show vendors with meaningful volume
ORDER BY applications DESC;
"""
funnel_vendor_df = run_query(sql_funnel_by_vendor)
funnel_vendor_df

In [ ]:
# Compute conversion rates per vendor
for col, numer in [
    ('start_rate', 'started'),
    ('submit_rate', 'submitted'),
    ('approve_rate', 'approved'),
    ('fund_rate', 'funded'),
]:
    if col == 'start_rate':
        denom = 'applications'
    elif col == 'submit_rate':
        denom = 'started'
    elif col == 'approve_rate':
        denom = 'submitted'
    else:
        denom = 'approved'
    funnel_vendor_df[col] = funnel_vendor_df[numer] / funnel_vendor_df[denom]

funnel_vendor_df[['vendor', 'applications', 'start_rate', 'submit_rate', 'approve_rate', 'fund_rate']]

In [ ]:
# Visualize funded rate by vendor
plt.figure(figsize=(10, 5))
sns.barplot(
    data=funnel_vendor_df.sort_values('fund_rate', ascending=False),
    x='vendor',
    y='fund_rate',
    palette='Blues_d'
)
plt.title("Funded Rate by Vendor")
plt.xlabel("Vendor")
plt.ylabel("Funded rate (funded / approvals)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Insight – Vendor and provider performance

- Vendors and provider partners differ not just in volume but in **quality** (approval and funded‑plan rates).  
- A provider or vendor with lower volume but high funded rate might deserve deeper partnership, better operational support, or more marketing budget.  
- Conversely, a high‑volume partner with poor downstream conversion can be targeted for **data quality** fixes (e.g., missing fields) or **targeting** and workflow improvements, or even deprioritized in the partnership mix.

In [ ]:
sql_channel_efficiency = """
WITH app_mkt AS (
    SELECT
        fa.application_id,
        fa.application_vendor,
        fa.channel,
        CASE WHEN fa.application_status = 'approved' THEN 1 ELSE 0 END AS is_approved,
        CASE WHEN fa.has_loan THEN 1 ELSE 0 END                        AS is_funded,
        fm.vendor          AS mkt_vendor,
        fm.channel         AS mkt_channel,
        fm.first_touch_cost_usd
    FROM fct_applications fa
    LEFT JOIN fct_marketing_attribution fm
        ON fa.application_id = fm.application_id
)
SELECT
    COALESCE(mkt_channel, channel) AS channel,
    COUNT(*)                        AS applications,
    SUM(is_approved)                AS approvals,
    SUM(is_funded)                  AS funded,
    SUM(first_touch_cost_usd)       AS spend_usd,
    SUM(first_touch_cost_usd) / NULLIF(SUM(is_funded), 0) AS cac_funded
FROM app_mkt
GROUP BY COALESCE(mkt_channel, channel)
ORDER BY applications DESC;
"""
channel_df = run_query(sql_channel_efficiency)
channel_df

In [ ]:
 plt.figure(figsize=(10, 5))
sns.barplot(
    data=channel_df.sort_values('cac_funded'),
    x='channel',
    y='cac_funded',
    palette='Greens_d'
)
plt.title("Customer Acquisition Cost (Funded) by Channel")
plt.xlabel("Channel")
plt.ylabel("CAC (spend / funded loans)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Insight – Marketing efficiency

- **CAC per funded plan** by channel (including digital channels and provider referrals) helps Growth decide where to scale or cut spend.  
- If a channel has a high funded rate but very high CAC, we can explore creative, landing‑page, or provider‑workflow optimizations before cutting it entirely.  
- These insights feed directly into the “vendor, provider & marketing performance” dashboards that a healthcare‑ or fintech data analyst would be expected to maintain.

## 6. Business Recommendations & Next Steps

Based on the modeling and analysis, here is how I would drive impact as a contract Data Analyst:

- **Tighten event tracking & schemas:**  
  Work with Engineering to ensure all key funnel steps (view, start, submit, approve, fund) are consistently tagged across web and app, and always tied to a stable `application_id` so funnels are reliable.

- **Establish a shared metrics layer:**  
  Promote `fct_applications`, `dim_loan`, and `fct_marketing_attribution` as the single source of truth for funnel and vendor metrics, documented in a central glossary accessible to Product, Ops, and Growth.

- **Formalize QA / UAT routines:**  
  Automate the validation queries (row counts, referential checks, vendor reconciliation) to run on every data refresh, with alerts if thresholds are breached, so issues are caught before stakeholders see dashboards.

- **Standardize vendor & channel dashboards:**  
  Create a canonical “Lending Funnel by Vendor & Channel” dashboard and a separate “Data Quality Monitor” dashboard to replace ad‑hoc, one‑off reports and enforce consistent definitions.

- **Partner with Product on experimentation:**  
  Use this metrics layer to define success metrics for UX and policy experiments (e.g., changes to underwriting criteria or application flow) and design dashboards that track experiment outcomes over time.

- **Roadmap to production‑grade analytics stack:**  
  As a follow‑up phase, implement a tool like dbt for modular SQL modeling and testing, and define data contracts with upstream teams so that any schema or tracking changes are coordinated rather than breaking downstream analytics.

---

This end‑to‑end case study shows how I would approach lending funnel analytics at a fintech like Splash: from raw, inconsistent datasets to a well‑documented, validated metrics layer and actionable dashboards that cross Engineering, Product, and Business stakeholders.
